In [1]:
import os.path
import pandas as pd
import json
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta


In [2]:
def connect_to_google_sheet(scopes, sheet_id, range_):
    if os.path.exists("Keyfile.json"):
        creds = service_account.Credentials.from_service_account_file('Keyfile.json')

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = (sheet.values().get(spreadsheetId=sheet_id, range=range_).execute())

    values = result.get('values')
    
    return values

In [3]:
def get_sheet():
    date = (datetime.now()+timedelta(days=0)).strftime('%Y-%m-%d')
    
    scopes = [""]

    sheet_id = ''
            
    range_ = (f'Projections for Date: {date}!A:P')

    values = connect_to_google_sheet(scopes, sheet_id, range_)
        
    values = [i for i in values if i.count('') == 0]

    columns = values[0]
    rows = values[1:]

    df = pd.DataFrame(rows, columns=columns)
    
    df = df[[
        'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Projected Total', 'Consensus Total', 'Projected Spread',
        'Consensus Spread', 'Bet Type', 'EV'
    ]]

    df.columns = [
       'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Projected Total', 'Consensus Total', 'Projected Spread',
        'Consensus Spread', 'Bet Type', 'TEV', 'SEV'
    ]
    
    return df

In [4]:
predictions = get_sheet()

In [5]:
predictions

,Home Team,Home Score,Away Team,Away Score,Projected Total,Consensus Total,Projected Spread,Consensus Spread,Bet Type,TEV,SEV
0,Auburn,85.1,Florida,77.5,162.6,155.5,7.6,5.5,over,4.70%,2.54%
1,Illinois,78.5,Wisconsin,75.4,153.9,148.5,3.1,3,over,3.92%,0.14%
2,VCU,66,Duquesne,65,131,131,1,1,under,0.00%,0.00%
3,UAB,75,Temple,70,145,145,5,5,under,0.00%,0.00%
4,Yale,73,Brown,65,138,138,8,8,under,0.00%,0.00%


In [6]:
predictions.to_json('preds.json', orient='records', indent=4) 

In [7]:
predictions.to_csv('preds.csv', index=False)